In [3]:
import numpy as np

from lib.io import load_stream
from lib.heartrate_zones import HeartrateZones
from lib.metrics import compute_metrics

user_id = '4a674085-61a1-4f39-83a6-c00d6870bf6b'
activity_id = 16323653057
health_metrics = {'max_heartrate': 196}

In [42]:
df = load_stream(activity_id, user_id)

In [43]:
df['dt'] = df['time'].diff().fillna(0)

## Test HR Model

In [5]:
heartrate = np.array([
    0.4,0.4,0.4,
    0.6,0.6,0.6,
    0.78,
    0.85,
    0.92,
    1.0
]) * health_metrics['max_heartrate']


In [6]:
heartrate_zones = HeartrateZones(health_metrics)

In [7]:
heartrate_zones.get_zone(heartrate)

array([0, 0, 0, 1, 1, 1, 2, 3, 4, 5])

In [8]:
heartrate_zones.get_zone(20)

0

In [9]:
heartrate_zones.get_zone(196 * 0.6)

1

In [10]:
heartrate_zones.get_zone(196 * 0.7999)

2

## Compute time in HR zones

In [58]:
def label_heartrate_zones(df, health_metrics):
    data = df.copy()
    heartrate_zones = HeartrateZones(health_metrics)
    data['heartrate_zone'] = heartrate_zones.get_zone(data['heartrate'])
    return data

def get_time_in_heartrate_zones(df):
    """seconds"""
    heartrate_zones = data.groupby('heartrate_zone')['dt'].sum().reindex(range(6)).fillna(0).to_dict()
    return {f'zone_{z}_time': s for z,s in heartrate_zones.items()}

In [59]:
df = label_heartrate_zones(df, health_metrics)

In [60]:
time_in_heartrate_zones = get_time_in_heartrate_zones(df)

In [61]:
time_in_heartrate_zones  # note: Z0 = below zone1 (e.g. <55% max HR)

{'zone_0_time': 6682.0,
 'zone_1_time': 6330.0,
 'zone_2_time': 42.0,
 'zone_3_time': 0.0,
 'zone_4_time': 0.0,
 'zone_5_time': 0.0}

In [6]:
metrics = compute_metrics(
    activity_id=activity_id,
    user_id=user_id,
    health_metrics=health_metrics
)

In [7]:
metrics

{'analyzed_at': '2025-11-15T05:48:56.869596',
 'uphill_heartrate': 118.27573424579413,
 'downhill_heartrate': 94.19685430463576,
 'zone_0_time': 6682.0,
 'zone_1_time': 6330.0,
 'zone_2_time': 42.0,
 'zone_3_time': 0.0,
 'zone_4_time': 0.0,
 'zone_5_time': 0.0}